In [1]:
!pip install pymysql --quiet
!pip install sqlalchemy --quiet
!pip install pyarrow --quiet


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
user = input("user: ")
password = input("password: ")
ip = input("ip: ")
key = input("key: ")

In [3]:
from sqlalchemy import create_engine
import pandas as pd

sql_uri = f'mysql+pymysql://{user}:{password}@{ip}:3306/playerdata'

engine = create_engine(sql_uri)
sql=f'''
select 
    HEX(AES_ENCRYPT(pl.name, "{key}")) as name, 
    pl.created_at, 
    pl.updated_at, 
    pl.possible_ban, 
    pl.confirmed_ban, 
    pl.label_id, 
    lb.label,
    case 
        when label_jagex = 2 then "banned"
        else null
    end as account_status,
    phd.*
from Players pl 
join playerHiscoreDataLatest phd on pl.id = phd.Player_id
join Labels lb on pl.label_id = lb.id
where pl.label_id <> 0'''
with engine.connect() as conn:
    df = pd.read_sql(sql, params=None ,con=conn)
df

In [ ]:
SKILLS = [
    "attack",
    "defence",
    "strength",
    "hitpoints",
    "ranged",
    "prayer",
    "magic",
    "cooking",
    "woodcutting",
    "fletching",
    "fishing",
    "firemaking",
    "crafting",
    "smithing",
    "mining",
    "herblore",
    "agility",
    "thieving",
    "slayer",
    "farming",
    "runecraft",
    "hunter",
    "construction",
]
MINIGAMES = [
    "league",
    "bounty_hunter_hunter",
    "bounty_hunter_rogue",
    "cs_all",
    "cs_beginner",
    "cs_easy",
    "cs_medium",
    "cs_hard",
    "cs_elite",
    "cs_master",
    "lms_rank",
    "soul_wars_zeal",
]
BOSSES = [
    "abyssal_sire",
    "alchemical_hydra",
    "barrows_chests",
    "bryophyta",
    "callisto",
    "cerberus",
    "chambers_of_xeric",
    "chambers_of_xeric_challenge_mode",
    "chaos_elemental",
    "chaos_fanatic",
    "commander_zilyana",
    "corporeal_beast",
    "crazy_archaeologist",
    "dagannoth_prime",
    "dagannoth_rex",
    "dagannoth_supreme",
    "deranged_archaeologist",
    "general_graardor",
    "giant_mole",
    "grotesque_guardians",
    "hespori",
    "kalphite_queen",
    "king_black_dragon",
    "kraken",
    "kreearra",
    "kril_tsutsaroth",
    "mimic",
    "nex",
    "nightmare",
    "phosanis_nightmare",
    "obor",
    "sarachnis",
    "scorpia",
    "skotizo",
    "Tempoross",
    "the_gauntlet",
    "the_corrupted_gauntlet",
    "theatre_of_blood",
    "theatre_of_blood_hard",
    "thermonuclear_smoke_devil",
    "tombs_of_amascut",
    "tombs_of_amascut_expert",
    "tzkal_zuk",
    "tztok_jad",
    "venenatis",
    "vetion",
    "vorkath",
    "wintertodt",
    "zalcano",
    "zulrah",
]
HISCORE_COLUMNS = ["total"] + SKILLS + MINIGAMES + BOSSES

In [ ]:
df[HISCORE_COLUMNS] = df[HISCORE_COLUMNS].replace(-1, 0)
df["total"] = df[SKILLS].sum(axis=1)

In [ ]:
from datetime import datetime 
now = datetime.now().strftime("%Y-%m-%d")
now

'2022-10-24'

In [ ]:
df.to_parquet(
    f'data/{now}_hiscore_data.parquet.gzip',
    compression='gzip'
)
